In [0]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [0]:
#combined training and testing sets to have entire dataset to ensure features are the
#same in both train and test sets
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'16N6CwZ16PBfQf1Jj2zEwSBq3bKpYi0gP'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')
downloaded = drive.CreateFile({'id':'1deEEIhUO5NJGebfWNfS0rzBtZhoPB8zh'})
downloaded.GetContentFile('test.csv')


#3. Read file as panda dataframe
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]    

In [11]:
for dataset in combine:
    dataset.loc[ dataset[' shares'] <= 950, ' shares'] = 0
    dataset.loc[(dataset[' shares'] > 950) & (dataset[' shares'] <= 1400), ' shares'] = 1
    dataset.loc[(dataset[' shares'] > 1400) & (dataset[' shares'] <= 2800), ' shares'] = 2
    dataset.loc[ dataset[' shares'] > 2800, ' shares'] = 3

train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/06/06/social-makeover...,581.0,9.0,645.0,0.523659,1.0,0.705722,9.0,4.0,1.0,...,0.033333,0.8,-0.249653,-0.4,-0.125,0.066667,0.033333,0.433333,0.033333,0
1,http://mashable.com/2013/03/18/gears-of-war-ju...,661.0,11.0,2104.0,0.376261,1.0,0.557325,25.0,3.0,11.0,...,0.100000,1.0,-0.275221,-0.8,-0.050,0.100000,0.000000,0.400000,0.000000,0
2,http://mashable.com/2014/09/17/ios-8-arrives-h...,113.0,10.0,540.0,0.580786,1.0,0.649852,30.0,2.0,1.0,...,0.033333,0.8,-0.300000,-0.8,-0.050,0.900000,0.500000,0.400000,0.500000,1
3,http://mashable.com/2014/09/22/twitter-app-how...,107.0,10.0,711.0,0.530670,1.0,0.668113,21.0,7.0,2.0,...,0.033333,0.8,-0.251282,-0.5,-0.050,0.100000,0.200000,0.400000,0.200000,0
4,http://mashable.com/2014/07/31/doctor-who-prem...,161.0,13.0,796.0,0.473418,1.0,0.624242,12.0,0.0,1.0,...,0.100000,0.6,-0.194753,-0.6,-0.050,0.400000,0.100000,0.100000,0.100000,2


In [0]:
X_train = train_df.drop(['url', ' shares'],axis=1)
Y_train = train_df[' shares']
X_test = test_df.drop(['url', ' shares'],axis=1)
Y_test = test_df[' shares']

In [0]:
logreg = LogisticRegression()
logreg.fit(X_train,Y_train)
Y_pred = logreg.predict(X_test)

In [17]:
import matplotlib.pyplot as plt
from sklearn import metrics

print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_log = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_log = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_log = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_log = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_log = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)
acc_log

[[1083  349  172  471]
 [ 851  383  212  534]
 [ 655  354  292  654]
 [ 475  234  274  936]]
             precision    recall  f1-score   support

          0       0.35      0.52      0.42      2075
          1       0.29      0.19      0.23      1980
          2       0.31      0.15      0.20      1955
          3       0.36      0.49      0.41      1919

avg / total       0.33      0.34      0.32      7929



ValueError: ignored

In [0]:
print(round(logreg.score(X_train,Y_train) * 100, 2))

33.3


In [0]:
coeff_df = pd.DataFrame(X_train.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by="Correlation", ascending=False)

,Feature,Correlation
0,n_tokens_title,1.315775e-04
25,kw_avg_avg,9.404174e-05
24,kw_max_avg,4.914224e-05
19,kw_avg_min,1.982332e-05
9,average_token_length,1.146844e-05
1,n_tokens_content,6.886253e-06
13,data_channel_is_bus,4.627502e-06
17,kw_min_min,2.897074e-06
39,LDA_02,1.931952e-06
40,LDA_03,1.784632e-06


In [0]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_svc = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_svc = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_svc = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_svc = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_svc = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)

[[   2 2054    0    0]
 [   1 1998    0    0]
 [   1 1954    0    0]
 [   0 1919    0    0]]
             precision    recall  f1-score   support

          0       0.50      0.00      0.00      2056
          1       0.25      1.00      0.40      1999
          2       0.00      0.00      0.00      1955
          3       0.00      0.00      0.00      1919

avg / total       0.19      0.25      0.10      7929



/homes/an325/da/lib64/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


ValueError: Target is multiclass but average='binary'. Please choose another average setting.

In [3]:
#K Nearest Neighbor

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_knn = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_knn = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_knn = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_knn = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_knn = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)

NameError: ignored

In [0]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_gaussian = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_gaussian = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_gaussian = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_gaussian = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_gaussian = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)

In [0]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_perceptron = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_perceptron = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_perceptron = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_perceptron = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_perceptron = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)

In [0]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_linear_svc = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_linear_svc = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_linear_svc = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_linear_svc = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_linear_svc = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)

In [0]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_sgd = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_sgd = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_sgd = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_sgd = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_sgd = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)

In [0]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_decision_tree = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_decision_tree = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_decision_tree = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_decision_tree = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_decision_tree = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)

In [0]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_random_forest = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_random_forest = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_random_forest = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_random_forest = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_random_forest = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)

In [18]:
# AdaBoost

adaboost = AdaBoostClassifier(n_estimators=100)
adaboost.fit(X_train, Y_train)
Y_pred = adaboost.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_adaboost = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_adaboost = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_adaboost = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_adaboost = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_adaboost = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)

[[1166  378  231  300]
 [ 663  441  477  399]
 [ 417  356  652  530]
 [ 339  234  513  833]]
             precision    recall  f1-score   support

          0       0.45      0.56      0.50      2075
          1       0.31      0.22      0.26      1980
          2       0.35      0.33      0.34      1955
          3       0.40      0.43      0.42      1919

avg / total       0.38      0.39      0.38      7929



ValueError: ignored

In [0]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Accuracy': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree],
    'Precision': [prec_svc, prec_knn, prec_log, 
              prec_random_forest, prec_gaussian, prec_perceptron, 
              prec_sgd, prec_linear_svc, prec_decision_tree],
    'Recall': [recall_svc, recall_knn, recall_log, 
              recall_random_forest, recall_gaussian, recall_perceptron, 
              recall_sgd, recall_linear_svc, recall_decision_tree],
    'F1': [f1score_svc, f1score_knn, f1score_log, 
              f1score_random_forest, f1score_gaussian, f1score_perceptron, 
              f1score_sgd, f1score_linear_svc, f1score_decision_tree],
    'AUC': [auc_svc, auc_knn, auc_log, 
              auc_random_forest, auc_gaussian, auc_perceptron, 
              auc_sgd, auc_linear_svc, auc_decision_tree]}, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
models.sort_values(by='Accuracy', ascending=False)

In [0]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(LogisticRegression(), X_train, Y_train, scoring='accuracy', cv=10)
print(scores)
print(scores.mean())

In [0]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(RandomForestClassifier(n_estimators=100), X_train, Y_train, scoring='accuracy', cv=10)
print(scores)
print(scores.mean())
acc_cv_random_forest = round(scores.mean() * 100, 2)
print(acc_cv_random_forest)